In [37]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import glob
import os
import metpy
import numpy as np
from datetime import datetime
import xarray as xr
import metpy.calc as mpcalc
from metpy.units import units
from metpy.calc import dewpoint_from_relative_humidity
from metpy.calc import specific_humidity_from_dewpoint
from pint import UnitRegistry
ureg = UnitRegistry()


In [20]:
# Function to extract epoch from the first few rows of the CSV file
def extract_epoch_from_header(file):
    # Read the first few lines to find the epoch (assuming it's in a comment or a header)
    with open(file, 'r') as f:
        skiprow = f.readline()
        skiprow = skiprow.strip().split()[0]
        for line in f:
            if "startdate:" in line.lower():  # Assuming the word 'epoch' is present in the line
                # Extract the epoch date (this assumes the date is the second item in the line)
                epoch = line.strip().split()[1]
                return int(skiprow)-1, epoch[0:4]+"-"+epoch[4:6]+"-"+epoch[6:8]+" "+epoch[8:10]+":"+epoch[10:12]+":"+epoch[12:14]
    return None  # Return None if no epoch is found

# Function to load and adjust time for each CSV
def load_and_adjust_time(file):
    # Try to extract the epoch from the header or a specific column
    nskiprows,epoch = extract_epoch_from_header(file)
   
    if epoch is None:
        raise ValueError(f"Could not find epoch for file: {file}")
   
    # Load the actual data (skipping header if necessary)
    df = pd.read_csv(file, skiprows=nskiprows,sep="\\s+")  # Adjust skiprows based on where the data starts
   
    # Convert 'time' column to a datetime, assuming it's in days
    # Adjust time column to start from the epoch time found in the file
    df['starttime_dt'] = pd.to_timedelta(df['starttime'], unit='D') + pd.Timestamp(epoch)
    #df = df[df.flag_rH != 0.999]
    #df.index = df["starttime_dt"]
    #data["starttime_dt"] = pd.to_datetime(data["starttime"],unit='D',origin=pd.Timestamp('2015-01-01 00:41:00'))
   
    # Return the adjusted DataFrame
    return df

In [ ]:
#set a working directory till NMVOC folder
working_dir = "C:\\Year1\\"
# Find all CSV files (change the path if nedeed)
rh_files = glob.glob(working_dir+"NMVOC\\Meteo_ebas_CMN\\Ebas_240920_1134\\*.nas")
print(rh_files)

# this function reads and adjust times for all CSV files
dfs = []
for file in rh_files:
    df = load_and_adjust_time(file)
    dfs.append(df)

# concatenates all DataFrames (based on time)
rh_df = pd.concat(dfs).sort_values('starttime_dt').reset_index(drop=True)

# Show combined DataFrame
print(rh_df)

In [39]:
# Function to extract epoch from the first few rows of the CSV file
def extract_epoch_from_header(file):
    # Read the first few lines to find the epoch (assuming it's in a comment or a header)
    with open(file, 'r') as f:
        skiprow = f.readline()
        skiprow = skiprow.strip().split()[0]
        for line in f:
            if "startdate:" in line.lower():  # Assuming the word 'epoch' is present in the line
                # Extract the epoch date (this assumes the date is the second item in the line)
                epoch = line.strip().split()[1]
                return int(skiprow)-1, epoch[0:4]+"-"+epoch[4:6]+"-"+epoch[6:8]+" "+epoch[8:10]+":"+epoch[10:12]+":"+epoch[12:14]
    return None  # Return None if no epoch is found

# Function to load and adjust time for each CSV
def load_and_adjust_time(file):
    # Try to extract the epoch from the header or a specific column
    nskiprows,epoch = extract_epoch_from_header(file)
   
    if epoch is None:
        raise ValueError(f"Could not find epoch for file: {file}")
   
    # Load the actual data (skipping header if necessary)
    df = pd.read_csv(file, skiprows=nskiprows,sep="\\s+")  # Adjust skiprows based on where the data starts
   
    # Convert 'time' column to a datetime, assuming it's in days
    # Adjust time column to start from the epoch time found in the file
    df['starttime_dt'] = pd.to_timedelta(df['starttime'], unit='D') + pd.Timestamp(epoch)
   
    # Return the adjusted DataFrame
    return df

In [ ]:
#set a working directory till NMVOC folder
working_dir = "C:\\Year1\\"
# Find all CSV files (change the path if nedded)
temperature_files = glob.glob(working_dir+"NMVOC\\Meteo_ebas_CMN\\Ebas_240920_1140\\*.nas")
print(temperature_files)

# this function reads and adjust times for all CSV files
dfs = []
for file in temperature_files:
    df = load_and_adjust_time(file)
    dfs.append(df)

# concatenates all DataFrames (based on time)
temperature_df = pd.concat(dfs).sort_values('starttime_dt').reset_index(drop=True)

# Show combined DataFrame
print(temperature_df)

In [ ]:
#set a working directory till NMVOC folder
working_dir = "C:\\Year1\\"
# Find all CSV files (change the path if nedded)
temperature_files = glob.glob(working_dir+"NMVOC\\Meteo_ebas_CMN\\Ebas_240923_1438\\*.nas")
print(temperature_files)

# this function reads and adjust times for all CSV files
dfs = []
for file in temperature_files:
    df = load_and_adjust_time(file)
    dfs.append(df)

# concatenates all DataFrames (based on time)
p_df = pd.concat(dfs).sort_values('starttime_dt').reset_index(drop=True)

# Show combined DataFrame
print(p_df)

In [ ]:
temperature_df

In [ ]:
#remove lines depending on flags
df=temperature_df
df ["RH"] = rh_df.rH
df ["flag_RH"] = rh_df.flag_rH
df ["flag_P"] = p_df.flag_P
df ["P"] = p_df.P
df = df[df.flag_T != 0.999]
df = df[df.flag_RH != 0.999]
df = df[df.flag_P != 0.999]
df = df[df.flag_T != 0.899]
df = df[df.flag_RH != 0.899]
df = df[df.flag_P != 0.899]
df = df[df.flag_T != 0.260]
df = df[df.flag_RH != 0.260]
df = df[df.flag_P != 0.260]
df = df[df.flag_T != 0.259]
df = df[df.flag_RH != 0.259]
df = df[df.flag_P != 0.259]
df = df[df.flag_T != 0.256]
df = df[df.flag_RH != 0.256]
df = df[df.flag_P != 0.256]
df.index = df["starttime_dt"]
#remove useless columns
df = df.drop(columns=["flag_P"])
df = df.drop(columns=["flag_RH"])
df = df.drop(columns=["flag_T"])
df = df.drop(columns=["starttime"])
df = df.drop(columns=["endtime"])
df

In [ ]:
#loop for rounding to the hours
df["starttime_dt"] = [i.round("H") for i in df["starttime_dt"]]
df

In [26]:
# helper columns with  hour, month, year
df['hour'] = df['starttime_dt'].dt.hour
df['month'] = df['starttime_dt'].dt.month
df['year'] = df['starttime_dt'].dt.year
df['date'] = df['starttime_dt'].dt.date

In [ ]:
#dewpoint from T and RH
dewpoint=dewpoint_from_relative_humidity(df["T"].values*units.degC, df["RH"].values*units.percent)
dewpoint

In [ ]:
#Specific humidity from temperature and dewpoint
SH=specific_humidity_from_dewpoint(df["P"].values*units.hPa, dewpoint).to('g/kg')
df ["SH"] = SH
df

In [ ]:
#calculate monthly mean, 'min', 'max', 'std', 'count' of SH
df['month'] = df['starttime_dt'].dt.month
SH_mean_hour=(df.groupby(['month']).agg({'SH': ['mean','min', 'max', 'std', 'count'] }))
SH_mean_hour

In [ ]:
#calculate monthly means each year
df.groupby([df["starttime_dt"].dt.year, "month"])["SH"].mean()


In [ ]:
#plot hourly mean
df.groupby(df["starttime_dt"].dt.hour)["SH"].mean().plot(marker="o",linewidth=0)

plt.xlabel("Time, hour");  # custom x label using Matplotlib

plt.ylabel("$Specific humidity (g/kg)$")

In [33]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Function to determine season
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [34]:
#season
seasons = [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1]
month_to_season = dict(zip(range(1,13), seasons))
month_to_season 
S=df.starttime_dt.dt.month.map(month_to_season) 
df['season']=S


In [ ]:
#means per season each year 
df.groupby(['year', 'season']).agg({'SH': ['mean','min', 'max', 'std', 'count']})
plot = df.groupby(['year', 'season']).agg({'SH': ['mean']}).plot(title="Seasonal mean specific humidity")#,marker="o",linewidth=0)



In [ ]:

#Extract year and season
df['year'] = df['date'].dt.year
df['season'] = df['date'].apply(get_season)

# Group by year and season and calculate mean value (or sum, etc.)
grouped = df.groupby(['season', 'year'])['SH'].mean().unstack()

# Reorder the seasons
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
grouped = grouped.reindex(season_order)

# Plot for different years
grouped.plot(kind='line', marker='o', figsize=(10, 6))

# Customize plot
plt.title('Value by Season for Different Years')
plt.ylabel('SH [g/kg]')
plt.xlabel('Season')
plt.xticks(range(len(season_order)), season_order)
plt.legend(title='Year')
plt.grid(True)

# Show plot
plt.show()